# Este notebook lo usaré para hacer el modelado del TFM

Referencias:
* https://medium.com/dunder-data/selecting-subsets-of-data-in-pandas-6fcd0170be9c
* https://medium.com/@juan.7655/cleansing-messy-excel-files-with-pandas-95a2ba3d8b2d
* https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/
* https://datatofish.com/convert-pandas-dataframe-to-list/
* https://www.geeksforgeeks.org/python-pandas-dataframe-insert/
* https://thispointer.com/python-pandas-drop-columns-in-dataframe-by-label-names-or-by-index-positions/

In [1]:
import pandas as pd
import math
import timeit

#path = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\" #Path para mi laptop en casa
path = "F:\\DATA_TFM\\CLEANDATA\\" #Path para mi PC en el trabajo

archivo = "df_best_hits_limpio.csv"
file_hits = path+archivo

archivo = "data_aafit_bbfit_muon.csv"
file_aafit = path+archivo

## Analizando los datos (dataset).

<font size=3>Después de todo el trabajo de preprocesamiento y limpieza, tenemo que, estos están por el momento divididos en dos partes.

* Dataframe 1, el cual contiene los hits, mismo que proviene del archivo llamado df_best_hits_limpios.csv
* Dataframe 2, el cual contiene la data de aafit, bbfit y el muon por frameID (evento), mismo que proviene del archivo data_aafit_bbfit_muon.csv
</font>

In [2]:
#Veamos como es el archivo del dataset relativo a los hits
df_hits = pd.read_csv(file_hits)

#En cargar tarda "bastante" alrededor de 4 min en la PC del trabajo (Core i5, 32GB RAM, Disco SSD.)

In [3]:
#Obtenemos información del dataframe
df_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63520301 entries, 0 to 63520300
Data columns (total 17 columns):
Unnamed: 0         int64
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
#hit               int64
lin_det            int64
piso_det           int64
mód_opt            int64
X'                 float64
Y'                 float64
Z'                 float64
azimut             float64
zenit              float64
tiempo             float64
amp                float64
frec_hits          float64
dtypes: float64(8), int64(9)
memory usage: 8.0 GB


In [23]:
#Mostramos las 10 primeras líneas para "ver" como son
df_hits.head(10)

,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,amp,frec_hits
0,25800,73557,0,2,76,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,296.559100,1.089169,327.587128
1,25800,73557,0,2,77,4,19,0,45.532615,9.219245,90.040108,0.197620,2.356193,137.146191,0.879543,327.587128
2,25800,73557,0,2,78,4,19,1,44.584223,9.539780,90.040108,2.292014,2.356193,153.189412,0.772948,281.085968
3,25800,73557,0,2,67,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,168.470446,1.070132,82.550049
4,25800,73557,0,2,68,4,20,0,45.484239,9.361442,104.567108,0.458220,2.356193,897.750991,0.985427,82.550049
5,25800,73557,0,2,69,4,20,1,44.485280,9.426792,104.567108,2.552615,2.356193,167.488297,4.492627,73.738098
6,25800,73557,0,2,70,4,20,2,44.928075,8.529054,104.567108,-1.636175,2.356193,171.860943,1.107187,89.874268
7,25800,73557,0,2,71,4,21,0,44.788333,9.655823,119.094108,1.882990,2.356193,201.236802,16.856011,60.100555
8,25800,73557,0,2,72,4,21,1,44.578323,8.677005,119.094108,-2.305800,2.356193,204.882482,3.327063,52.337646
9,25800,73557,0,2,73,4,21,2,45.530937,8.984459,119.094108,-0.211405,2.356193,1035.010812,0.580996,61.416626


In [20]:
#Es extraño pero tenemos una columnna unnamed la cual eliminaré, insisto es extraño puesto que en la extracción no se mostró.
#No sé si cuando se grabó a disco algo cambió.

df_hits = df_hits.drop([df_hits.columns[0]], axis='columns')

In [21]:
df_hits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63520301 entries, 0 to 63520300
Data columns (total 16 columns):
runID              int64
frameID            int64
trigger_counter    int64
interactionID      int64
#hit               int64
lin_det            int64
piso_det           int64
mód_opt            int64
X'                 float64
Y'                 float64
Z'                 float64
azimut             float64
zenit              float64
tiempo             float64
amp                float64
frec_hits          float64
dtypes: float64(8), int64(8)
memory usage: 7.6 GB


In [25]:
df_hits.sample(10)

,runID,frameID,trigger_counter,interactionID,#hit,lin_det,piso_det,mód_opt,X',Y',Z',azimut,zenit,tiempo,amp,frec_hits
62782667,85370,145642,1431,1,124,4,12,1,44.982226,8.960608,-11.664434,-1.541541,2.356193,-614.864546,1.385682,270.481110
60735988,84550,291378,710,2,184,9,19,1,-93.859173,9.057638,90.148566,0.693200,2.356193,-448.769866,0.864472,256.977081
28309445,57370,79452,1624,2,182,5,21,1,-57.848145,78.815537,119.021566,0.385280,2.356193,-297.879331,0.360747,77.075958
36779424,62100,53913,2666,2,136,9,11,0,-94.265366,8.111578,-25.970434,-1.504405,2.356193,1408.350233,1.098281,52.108765
60984252,84630,111210,1027,1,97,8,21,2,11.850980,-39.340843,119.886566,-0.692785,2.356193,116.103090,24.237271,51.174164
47095849,69370,29994,682,1,272,1,10,2,4.582201,96.176342,-49.279434,-1.169144,2.356193,734.745564,5.828434,66.833496
58749529,83750,10294,939,2,104,4,24,1,45.483717,9.282859,162.666566,-0.457871,2.356193,492.694216,0.861403,54.073334
34829646,61090,55830,4256,2,208,9,10,1,-93.784372,8.434716,-40.487434,-0.454290,2.356193,2831.700160,0.579883,52.909851
55220200,82120,284698,1459,1,142,8,13,0,11.635656,-39.502236,3.756566,-1.162735,2.356193,-707.211965,0.684769,40.054321
19233376,50030,28461,794,1,112,4,19,2,44.780179,8.990805,90.037566,-1.896775,2.356193,217.935494,0.871770,63.076019


In [5]:
#Contrastando con el dataframe obtenido para el punto 3.1 del email de Salva, abrimos el segundo dataframe
df_aafit = pd.read_csv(file_aafit)

In [6]:
#Obtenemos la info del segundo dataframe
df_aafit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021092 entries, 0 to 1021091
Data columns (total 14 columns):
runID              1021092 non-null int64
frameID            1021092 non-null int64
trigger_counter    1021092 non-null int64
interactionID      1021092 non-null int64
aafit_azimut       1021092 non-null float64
aafit_zenit        1021092 non-null float64
aafit_lambda       1021092 non-null float64
aafit_beta         1021092 non-null float64
bbfit_azimut       1021092 non-null float64
bbfit_zenit        1021092 non-null float64
bbfit_quality      1021092 non-null float64
muon_azimut        1021092 non-null float64
muon_zenit         1021092 non-null float64
muon_energy        1021092 non-null float64
dtypes: float64(10), int64(4)
memory usage: 109.1 MB


In [30]:
#Mostramos las 10 primeras líneas del segundo dataframe
df_aafit[df_aafit['runID']==25800].shape

df_hits[df_hits['runID']==25800].shape

(479, 14)

In [15]:
temp = df_aafit['runID'].unique()

In [16]:
len(temp)

632

In [17]:
temp
#Esto lo que me dice es que falta generar la data completa de TODOS los archivos
#usando el notebook GENERA_DF_AAFIT_BBFIT-FINAL

array([25800, 25880, 25920, 25930, 25990, 26010, 26030, 26110, 26160,
       26190, 26230, 26270, 26340, 26470, 26480, 26540, 26640, 26660,
       26710, 26720, 26770, 26800, 26810, 26820, 26830, 26870, 26920,
       26940, 26970, 27080, 27090, 27100, 27130, 27170, 27180, 27230,
       27270, 27290, 27440, 27460, 27470, 27550, 27580, 27620, 27640,
       27660, 27720, 27750, 27760, 27830, 27890, 27900, 27980, 27990,
       28070, 28110, 28280, 28340, 28350, 28450, 28460, 28510, 28570,
       28620, 28630, 28650, 28720, 28730, 28770, 28780, 28790, 28850,
       28860, 28870, 28950, 28970, 28980, 28990, 29040, 29050, 29060,
       29100, 29110, 29120, 29160, 29180, 29190, 29220, 29250, 29270,
       29280, 29320, 29330, 29350, 29360, 29370, 29420, 29440, 29480,
       29490, 29500, 29540, 29580, 29600, 29740, 29760, 29790, 29810,
       29820, 29830, 29840, 29910, 29920, 29980, 30020, 30030, 30060,
       30070, 30080, 30090, 30100, 30110, 30120, 30140, 30150, 30160,
       30170, 30210,